# Model classification I – building blocks

<div style="text-align: center; font-style: italic;">

“A problem well stated is a problem half-solved.”

</div>

<div style="text-align: right;">

— Charles F. Kettering

</div>